In [1]:
import hid
import time

VID = 0x0403
PID = 0x6030
INTERFACE = 1

In [2]:
# Tady se nalezne spravny HID zarizeni dle VID/PID a vybere se UART interface

devices = hid.enumerate()

for d in devices:
    if d['vendor_id'] == VID and d['product_id'] == PID:
        print("Nalezeno zařízení:", d)

        if d['interface_number'] == INTERFACE:
            try:
                device = hid.device()
                device.open_path(d['path'])
            finally:
                print("Vybrane zarizeni", device)
                #device.close()

Nalezeno zařízení: {'path': b'1-2:1.0', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 0}
Nalezeno zařízení: {'path': b'1-2:1.1', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 1}
Vybrane zarizeni <hid.device object at 0x7f3556ac7dd0>


In [3]:
print(d['path'])

b'1-2:1.1'


In [ ]:
# Otevrit HID zarizeni

#device = hid.device()
#device.open_path(d['path'])

In [ ]:
# Tohle přeskočit :) 

device.close()

In [4]:
# Vypis zakladni parametry o zarizeni 
try:
    print(f'Device manufacturer: {device.get_manufacturer_string()}')
    print(f'Product: {device.get_product_string()}')
    print(f'Serial Number: {device.get_serial_number_string()}')

    device.set_nonblocking(1)

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)


Device manufacturer: FTDI
Product: FT260
Serial Number: Љ


In [15]:
# Smycka, ktera vycita RX buffer u FTDI

try:
    while True:
        # Čtení dat
        data = device.read(64)  # Přečte až 64 bajtů dat
        if data:
            print("Přijatá data:", [hex(x) for x in data[2:2+data[1]]], data)
            #device.write(data[2])

        time.sleep(0.05)  # Krátká pauza pro snížení zatížení procesoru

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)
#finally:
#    # Uzavření zařízení
#    #device.close()


Přijatá data: ['0x43'] [240, 1, 67, 10, 107, 46]
Přijatá data: ['0x76', '0x61', '0x6b', '0x2e'] [240, 4, 118, 97, 107, 46]
Přijatá data: ['0x2e', '0x2e', '0xd', '0xa'] [240, 4, 46, 46, 13, 10]
Přijatá data: ['0x23'] [240, 1, 35, 46, 13, 10]
Přijatá data: ['0x43'] [240, 1, 67, 46, 13, 10]
Přijatá data: ['0x76'] [240, 1, 118, 46, 13, 10]
Přijatá data: ['0x61', '0x6b', '0x2e', '0x2e', '0x2e', '0xd', '0xa'] [241, 7, 97, 107, 46, 46, 46, 13, 10, 10]
Přijatá data: ['0x23'] [240, 1, 35, 107, 46, 46]
Přijatá data: ['0x43'] [240, 1, 67, 107, 46, 46]
Přijatá data: ['0x76', '0x61', '0x6b', '0x2e', '0x2e', '0x2e', '0xd', '0xa'] [241, 8, 118, 97, 107, 46, 46, 46, 13, 10]
Přijatá data: ['0x23'] [240, 1, 35, 97, 107, 46]
Přijatá data: ['0x43'] [240, 1, 67, 97, 107, 46]
Přijatá data: ['0x76', '0x61', '0x6b', '0x2e', '0x2e', '0x2e', '0xd', '0xa'] [241, 8, 118, 97, 107, 46, 46, 46, 13, 10]
Přijatá data: ['0x23'] [240, 1, 35, 97, 107, 46]
Přijatá data: ['0x43'] [240, 1, 67, 97, 107, 46]
Přijatá data: ['0

KeyboardInterrupt: 

In [24]:
# Smycka, ktera vycita RX buffer u FTDI

try:
    while True:
        # Čtení dat
        data = device.read(500)  # Přečte až 64 bajtů dat
        if data:
            print(bytes(data[2:2+data[1]]).decode('utf-8'))
            #device.write(data[2])

        time.sleep(0.05)  # Krátká pauza pro snížení zatížení procesoru

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)
#finally:
#    # Uzavření zařízení
#    #device.close()


$
HIST,29
,
$
HIST,30
,


KeyboardInterrupt: 

Tohle je funkcni blok, ktery posle obsah promenne 'zprava' po seriove lince. 

In [ ]:
# Poslat UART zpravu do arduina

zprava = b'\n\r Ahoj Testovaci zprava'
l = len(zprava)
payload = bytes([0xFE, l]) + zprava
print(l, payload)
device.write(payload)

In [ ]:
device.read(100)

In [ ]:
help(device.get_feature_report)

In [ ]:
d = device.get_feature_report(0xA1, 100)
da = [hex(x) for x in d]
print(len(d))
print(da)

print('flow_ctrl: {}'.format(d[2]))

In [ ]:
# Vypsani aktualniho nastaveni TF260 cipu

d = device.get_feature_report(0xE0, 100)
da = [hex(x) for x in d]
print(len(d))
print(da)
print("")
print('flow_ctrl: {}'.format(d[1]))
print('baud_rade: {}'.format([hex(x) for x in d[2:6]]))
print('data_bit: {}'.format(d[6]))
print('parity: {}'.format(d[7]))
print('stop_bit: {}'.format(d[8]))

In [ ]:
device.send_feature_report([0xA1, 0x41, 0x3, 0x00, 0x4B, 0x00, 0x00, 0x08, 0x01, 0x01, 0x00])

# Nastavení baudrate

In [8]:
# Timto se nastavi baudrate na 9600
device.send_feature_report([0xA1, 0x42, 0x80, 0x25, 0x00, 0x00])

6

In [10]:
br = 115200
brb = (br).to_bytes(4, byteorder="little")
print(br, brb)
payload = [0xA1, 0x42] + list(brb)
print("payload:", [hex(i) for i in payload])
device.send_feature_report(payload)

115200 b'\x00\xc2\x01\x00'
payload: ['0xa1', '0x42', '0x0', '0xc2', '0x1', '0x0']


6

# Nastavení funkce LEDek

In [11]:
# Vypnuti LEDEk (resp. prepnuti do RX/TX modu)

device.send_feature_report([0xA1, 0x08, 0x04])  # Nastavit GPIOA do TX_LED
device.send_feature_report([0xA1, 0x09, 0x05])  # Nastavit GPIOG do RX_LED

3

# Ovládání DTR pinu, reset arduina

In [12]:
device.send_feature_report([0xB0, 0x00, 0x00, 0b00100000, 0b00100000])
time.sleep(0.1)
device.send_feature_report([0xB0, 0x00, 0x00, 0b00000000, 0b00100000])
#time.sleep(0.1)

5

# Vyčtení dat z UARTu

tvořeno pro programátor, není asi plně funkční...

In [ ]:
def read(size=1):
    data_total = []
    for x in range(100):
        data = device.read(100)
        print(x, data)
        if data:
            if len(data):
                print(f"RECV [{size}]< ", [hex(x) for x in data])
                print(f"RECV [{size}]< ", [hex(x) for x in data[2:2+data[1]]])
                data_total += data[2:2+data[1]]
        if len(data_total) >= size:
            print("Done", len(data_total))
            return data_total
        time.sleep(0.05)
    return []
    

In [ ]:
print([hex(a) for a in read(30)])

# Write to UART 
tento blok funguje, pošle po UARTu string.

Vstupní parametr mají být typu `bytes`, např: `b'aaa'`

In [ ]:
def write(data):
    #print("SEND> ", [hex(x) for x in data] )
    data = bytes(data)
    l = len(data)
    payload = bytes([0xFE, l]) + data
    print(l, payload)
    device.write(payload)

In [ ]:
#reset arduino
device.send_feature_report([0xB0, 0x00, 0x00, 0b00100000, 0b00100000])
time.sleep(0.1)
device.send_feature_report([0xB0, 0x00, 0x00, 0b00000000, 0b00100000])

write(b'\x30\x20')
time.sleep(0.25)
read(10)